- repr(), bytes() 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성자 구현
- format() 내장 함수와 str.format() 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- __slots__를 이용해서 메모리 절약하기

### 9.1 객체 표현
- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환한다.
- str() : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환한다.

### 9.2 벡터 클래스의 부활

In [92]:

from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    

In [93]:
v1 = Vector2d(3,4)
print (v1.x, v1.y)
x,y = v1
print (x, y)
print (v1)

Vector2d(3.0, 4.0)
v1_clone = eval(repr(v1))
print (v1 == v1_clone)
print (v1)
octets = bytes(v1)
print (octets)
print (abs(v1))
print (bool(v1), bool(Vector2d(0, 0)))

3.0 4.0
3.0 4.0
(3.0, 4.0)
True
(3.0, 4.0)
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
5.0
True False


### 9.3 대안 생성자

In [94]:
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:].cast(typecode))
    return cls(*memv)

### 9.4 @classmethod와 @staticmethod

In [95]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

print (Demo.klassmeth())
print (Demo.klassmeth('spam'))
print (Demo.statmeth())
print (Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


### 9.5 포맷된 출력

In [96]:
br1 = 1/2.43 # 브라질 레알을 미국 달러로 바꾸는 환율
print (br1)
print (format(br1, '0.4f'))
print ('1 BRL = {rate:0.2f} USD'.format(rate=br1))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


In [97]:
print (format(42, 'b'))
print (format(2/3, '.1%'))

101010
66.7%


In [98]:
from datetime import datetime
now = datetime.now()
print (format(now, '%H:%M:%S'))
print ("It's now {:%I:%M: %p}".format(now))

23:08:43
It's now 11:08: PM


In [99]:
v1 = Vector2d(3,4)
format(v1)
#print (format(v1, '.3f'))

'(3.0, 4.0)'

In [100]:
v1 = Vector2d(3,4)
print (format(v1))
print (format(v1, '.2f'))
print (format(v1, '.3e'))

(3.0, 4.0)
(3.00, 4.00)
(3.000e+00, 4.000e+00)


In [101]:
print (format(Vector2d(1,1), 'p'))
print (format(Vector2d(1,1), '.3ep'))
print (format(Vector2d(1,1), '0.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


### 9.6 해시 가능한 Vector2d

In [102]:
v1 = Vector2d(3,4)
#hash(v1)
print (v1.x, v1.y)
v1.x = 7
print (v1.x, v1.y)

3.0 4.0
7 4.0


In [106]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    @property
    def x(self):
        return self.__x
    @property
    def y(self):
        return self.__y
    
    def __hash__(self):
        return hash(self.x)^hash(self.y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [108]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1, 4.2)
print (hash(v1))
print (hash(v2))
set([v1, v2])

7
384307168202284039


{Vector2d(3.1, 4.2), Vector2d(3.0, 4.0)}